In [1]:
import pandas as pd
from datetime import datetime

In [2]:
hdi_df = pd.read_csv('../data-sources/Human Development Index - Full.csv')
imf_gdp_df = pd.read_csv('../data-sources/imf-gdp.csv')

min_year = 2015
current_datetime = datetime.now()
current_year = current_datetime.year


In [7]:
column_list = [col for col in hdi_df.columns if col.startswith('Human Development Index (') and (int(col.replace(')', '').replace('Human Development Index (', '')) >= min_year) and (int(col.replace(')', '').replace('Human Development Index (', '')) <= current_year)]
column_list = ['Country'] + column_list

hdi_clean_df = hdi_df[column_list].dropna()
clean_df_columns = hdi_clean_df.columns
clean_df_columns = [col.replace(')', '').replace('Human Development Index (', '') for col in clean_df_columns]
hdi_clean_df.columns = clean_df_columns
hdi_clean_df.reset_index(drop=True, inplace=True)
hdi_clean_df = pd.melt(hdi_clean_df, id_vars=['Country'], var_name='Year').sort_values(by=['Country', 'Year'])
hdi_clean_df

,Country,Year,value
0,Afghanistan,2015,0.478
190,Afghanistan,2016,0.481
380,Afghanistan,2017,0.482
570,Afghanistan,2018,0.483
760,Afghanistan,2019,0.488
...,...,...,...
569,Zimbabwe,2017,0.594
759,Zimbabwe,2018,0.602
949,Zimbabwe,2019,0.601
1139,Zimbabwe,2020,0.600


In [9]:
imf_gdp_df.rename(columns={'GDP, current prices (Billions of U.S. dollars)': 'Country'}, inplace=True)
column_list = ['Country'] + [col for col in imf_gdp_df.columns if col != 'Country' and (int(col) >= min_year and int(col) <= current_year)]
column_list
clean_imf_gdp_df = imf_gdp_df[column_list].dropna()
clean_imf_gdp_df = clean_imf_gdp_df[clean_imf_gdp_df.isin(['no data']).any(axis=1) == False]
clean_imf_gdp_df.reset_index(drop=True, inplace=True)
clean_imf_gdp_df = pd.melt(clean_imf_gdp_df, id_vars=['Country'], var_name='Year').sort_values(by=['Country', 'Year'])
clean_imf_gdp_df

,Country,Year,value
208,ASEAN-5,2015,2177.95
429,ASEAN-5,2016,2284.18
650,ASEAN-5,2017,2462.823
871,ASEAN-5,2018,2631.544
1092,ASEAN-5,2019,2781.979
...,...,...,...
1293,Zimbabwe,2020,26.878
1514,Zimbabwe,2021,36.016
1735,Zimbabwe,2022,31.291
1956,Zimbabwe,2023,32.216


In [11]:
hdi_gdp_combined_df = pd.merge(hdi_clean_df, clean_imf_gdp_df, on=['Country', 'Year'], how='inner')
hdi_gdp_combined_df

,Country,Year,value_x,value_y
0,Albania,2015,0.795,11.389
1,Albania,2016,0.798,11.862
2,Albania,2017,0.802,13.053
3,Albania,2018,0.806,15.157
4,Albania,2019,0.810,15.399
...,...,...,...,...
1129,Zimbabwe,2017,0.594,22.046
1130,Zimbabwe,2018,0.602,37.065
1131,Zimbabwe,2019,0.601,26.033
1132,Zimbabwe,2020,0.600,26.878


In [12]:
hdi_gdp_combined_df.rename(columns={'value_x': 'HDI', 'value_y': 'GDP'}, inplace=True)
hdi_gdp_combined_df